In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import shutil
import random
import cv2
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
sns.set(rc={'figure.figsize':(10,6)})
sns.set(font_scale=1.3)
plt.style.use('fivethirtyeight')


!pip install -U -q pyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/gdrive')


!pip install tensorflow

!pip install -q keras

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation,Input, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD, RMSprop
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
! pip install split-folders

In [ ]:
import splitfolders

In [ ]:
input_folder = "/content/gdrive/MyDrive/Colab Notebooks/data1"
output = "/content/gdrive/MyDrive/Colab Notebooks/data2"

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.7,0.3))

Copying files: 5631 files [00:30, 184.42 files/s]


In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Colab Notebooks/data2/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

Found 3941 images belonging to 4 classes.


In [ ]:
# test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale = 1./255)
# test_set = test_datagen.flow_from_directory('/working/dataset/test',
#                                             target_size = (224, 224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')

In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale = 1./255)
val_set = val_datagen.flow_from_directory('/content/gdrive/MyDrive/Colab Notebooks/data2/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 1690 images belonging to 4 classes.


In [ ]:
early = EarlyStopping(monitor='val_loss',patience=5)
reduce = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, mil_lr=0.001)

In [ ]:
output_classes = 4
batch_size = 32
epochs = 10
image_shape=(224,224,3)
nb_train_samples = 3941
nb_validation_samples = 1690
# nb_test_samples = 564

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, pooling='avg', input_shape=image_shape)
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

model = Model(inputs=base_model.input,outputs=prediction)

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
adam_opt=Adam(lr=0.00001, beta_1=0.6, beta_2=0.9, amsgrad=True)
model.compile(optimizer= adam_opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import time
print("Training started it takes few minutes")
start_time = time.time()

history = model.fit(
  training_set,
  steps_per_epoch = nb_train_samples // batch_size,
  epochs = epochs,
  validation_data = val_set,
  validation_steps = nb_validation_samples // batch_size,
  callbacks = [early, reduce])

print("successfully completed the traing session")
print("--- took %d:%.2d minutes ---" % divmod(time.time() - start_time, 60))

Training started it takes few minutes


NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Acc', 'Test Acc'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Test Loss'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate_generator(val_set, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

Validation Score:  0.24223266541957855
Validation Accuracy:  0.9606249928474426


In [ ]:
# accuracy_score = model.evaluate_generator(test_set, steps=50)
# print("Accuracy Score:", accuracy_score[0])
# print("Accuracy:",accuracy_score[1])

In [ ]:
_, acc = model.evaluate(val_set, verbose=1)
print('> %.3f' % (acc * 100.0))

In [ ]:
model.save("Test" + '.h5')

### Model Evaluation

The evaluate_model function loads a trained model from a saved file and evaluates it using the validation generator. It calculates overall metrics (accuracy, precision, recall, F1-score) and class-specific metrics. These metrics provide a detailed view of model performance.

In [ ]:
def evaluate_model(saved_model_path, validation_generator):
    # Load the saved model
    loaded_model = tf.keras.models.load_model(saved_model_path)

    # Evaluate the model on the test set
    test_loss, test_accuracy = loaded_model.evaluate(validation_generator)
    print('Test Loss:', test_loss)
    print('Test Accuracy:', test_accuracy)

    # Perform thorough evaluation
    y_true = validation_generator.classes
    y_pred = loaded_model.predict(validation_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Calculate overall metrics
    accuracy = metrics.accuracy_score(y_true, y_pred_classes)
    precision = metrics.precision_score(y_true, y_pred_classes, average='weighted')
    recall = metrics.recall_score(y_true, y_pred_classes, average='weighted')
    f1_score = metrics.f1_score(y_true, y_pred_classes, average='weighted')

    print('Overall Metrics:')
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1-Score:', f1_score)

    # Calculate metrics per class
    class_metrics = metrics.classification_report(y_true, y_pred_classes, target_names=validation_generator.class_indices.keys())
    print('\nMetrics per Class:')
    print(class_metrics)

In [ ]:
import sklearn.metrics as metrics
saved_model_path = '/content/gdrive/MyDrive/Colab Notebooks/Test.h5'

evaluate_model(saved_model_path, val_set)
# plot_training_history(history)

## Build a function to predict random images from the internet
### Predicting on New Data
The predict_images_from_urls function predicts the class of new images provided via URLs. It fetches the images, preprocesses them, and feeds them into the model for prediction. The function also displays the images along with their predicted class. This function allows you to easily use the model to make predictions on new, unseen data.

In [ ]:
def predict_images_from_urls(model, classes, image_urls, target_size):
    num_images = len(image_urls)
    num_rows = (num_images + 3) // 4

    fig, axs = plt.subplots(nrows=num_rows, ncols=4, figsize=(12, 6))

    for i, image_url in enumerate(image_urls):
        row = i // 4
        col = i % 4

        # Download the image from the URL
        response = requests.get(image_url)

        if response.status_code == 200:
            try:
                image = Image.open(BytesIO(response.content))
                image = image.convert("RGB")  # Convert to RGB format

                # Resize the image
                image = image.resize(target_size)

                # Preprocess the image
                image_array = np.array(image) / 255.0
                image_array = np.expand_dims(image_array, axis=0)

                # Make the prediction
                prediction = model.predict(image_array)
                predicted_class_index = np.argmax(prediction)

                # Create a reverse dictionary of classes
                reverse_classes = {v: k for k, v in classes.items()}

                # Get the predicted class label
                predicted_class = reverse_classes[predicted_class_index]

                # Display the image and predicted class
                axs[row, col].imshow(image)
                axs[row, col].axis('off')
                axs[row, col].set_title(f"Predicted Class: {predicted_class}")
            except (UnidentifiedImageError, ImageDecompressionBombError) as e:
                print(f"Error processing image: {image_url}\n{e}")

    # Remove any empty subplots
    for i in range(num_images, num_rows * 4):
        row = i // 4
        col = i % 4
        fig.delaxes(axs[row, col])

    fig.tight_layout()
    plt.show()

In [ ]:
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO

In [ ]:
# Example list of image URLs
image_urls = [
    'https://previews.123rf.com/images/vampy1/vampy12007/vampy1200700021/151785736-satellite-view-of-namibia-desert-landscape-and-mountains-nature-and-aerial-view-global-warming.jpg',
    'https://thumbs.dreamstime.com/b/desert-land-satellite-photo-topography-aerial-top-view-earth-surface-as-abstract-background-nature-pattern-southern-sand-229623767.jpg',
    'https://cdn.mos.cms.futurecdn.net/Fry8CGJGgP6vgXBupGTqwT.jpg',
    'https://miro.medium.com/v2/resize:fit:1000/0*59yUF7X3jRrcrUnu.jpg',
    'https://media.rff.org/images/resized_erie_oli_2015209_lrg_40.2e16d0ba.fill-1270x1120.jpg',
    'https://img.rasset.ie/0014aa0a-500.jpg',
    'https://www.nasa.gov/images/content/624724main_pia15399-673.jpg',
    'https://cdn.mos.cms.futurecdn.net/aa3h9RKfsPcS8sPwyzSgu9-320-80.jpg',
]

# Load the saved custom model and classes
loaded_model = tf.keras.models.load_model(saved_model_path)
classes = {'cloudy':0, 'desert':1, 'green_area':2, 'water':3}
target_size = (224, 224)

# Predict and display the images from the URLs
predict_images_from_urls(loaded_model, classes, image_urls, target_size)